In [1]:
!pip install surprise
!pip install scikit-surprise

In [2]:
%%time
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, coo_matrix
from tqdm.auto import tqdm
from collections import Counter
from surprise import Dataset, Reader
import time
from surprise import KNNWithMeans

c:\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: total: 750 ms
Wall time: 1.49 s


In [3]:
df_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\track_artists.csv"
train_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\train"
test_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\test"
pred_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\surprize_artist_KNNWithMeans_target"
target_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\surprize_artist_KNNWithMeans_pred"

In [4]:
def calc_mrr(predict, answer):
    for i in range(len(predict)):
        if predict[i] == answer:
            return 1. / (i + 1)
    return 0

max_prediction_len = 100

def calc_score(target_path, predict_path):
    with open(target_path) as f:
        y_true = [int(x.strip()) for x in f.readlines()]

    with open(predict_path) as f:
        y_pred = [[int(x) for x in line.strip().split(' ')] for line in f.readlines()]

    mrr_score = 0
    for (pred, answer) in zip(y_pred, y_true):
        if len(pred) > max_prediction_len:
            raise ValueError('$maximum prediction length is {}, got {}$'.format(max_prediction_len, len(y_pred[i])))
        mrr_score += calc_mrr(pred, answer)

    print(f"MRR@100 = {(mrr_score / len(y_true)):.4f}")

In [5]:
%%time
df = pd.read_csv(df_path)
df.index = df['trackId']

CPU times: total: 78.1 ms
Wall time: 54.9 ms


In [6]:
%%time

list_coord = []
target_val = []
np.random.seed(42)  
val_id = np.random.choice(range(10000), size=1000, replace=False)
val_id.sort()
all_artist = set()
ls_artists = []

with open(train_path) as f:
    lines = f.readlines()
    idx = 0 
    for line in tqdm(lines):
        tracks = line.strip().split(' ')
        tracks_arr = np.array(tracks)
        tracks_arr = tracks_arr.astype(np.int32)
        artist_arr = np.array([df.at[track, 'artistId'] for track in tracks_arr])
        if idx in val_id:
            target_val.append(artist_arr[-1])
            artist_arr = artist_arr[:-1]
        ls_artists.extend(artist_arr)
        all_artist = all_artist.union(set(artist_arr))
        artist_cnt = Counter(artist_arr)
        list_coord_user = [(idx, i[0], i[1]) for i in artist_cnt.items()]
        list_coord.extend(list_coord_user)
        idx += 1
        if idx > 10000:
            break

  1%|▋                                                                        | 10000/1160084 [00:11<21:46, 880.60it/s]

CPU times: total: 12.5 s
Wall time: 12.4 s


In [7]:
%%time

data = pd.DataFrame(list_coord, columns=['uid', 'iid', 'rating_row'])
data['rating_max'] = data.rating_row / data.groupby('uid')['rating_row'].transform('max')
data['rating_part'] = data.rating_row / data.groupby('uid')['rating_row'].transform('sum')

CPU times: total: 2.62 s
Wall time: 2.63 s


# Baseline

In [8]:
all_artist_cnt = [i[0] for i in Counter(ls_artists).most_common(100)]

result = [' '.join(map(str, all_artist_cnt)) + '\n' for _ in val_id]

with open(pred_path, 'w') as f:
    f.writelines(result)

with open(target_path, 'w') as f:
    for i in target_val:
        f.write(f'{i}\n')

calc_score(target_path, pred_path)

MRR@100 = 0.0144


# RATING_MAX

In [9]:
%%time

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(data[['uid', 'iid', 'rating_max']], reader)
trainset = dataset.build_full_trainset()

CPU times: total: 547 ms
Wall time: 546 ms


## KNNWithMeans

In [11]:
def optimize(k=40, 
             min_k=1,
             user_based=True,
             name="pearson_baseline",
             verbose=True):

    start_time = time.time()
    
    sim_options = {
        "name": name,
        "user_based": user_based
    }
    
    algo = KNNWithMeans(k=k, min_k=min_k, sim_options=sim_options, verbose=True)
    
    algo.fit(trainset)
    
    print("--- fit: %s seconds ---" % (time.time() - start_time))
    
    pred = []
    for uid in tqdm(val_id):
        pred_user = []
        for iid in all_artist:
            score = algo.predict(uid=uid, iid=iid).est
            pred_user.append((iid, score))
        pred.append(pred_user)

    res = []
    for user in tqdm(pred):
        ls = sorted(user, key=lambda x: x[1])
        temp = [i[0] for i in ls[-100:]]
        temp.reverse()
        res.append(temp)

    result = [' '.join(map(str, i)) + '\n' for i in res]


    with open(pred_path, 'w') as f:
        f.writelines(result)

    with open(target_path, 'w') as f:
        for i in target_val:
            f.write(f'{i}\n')

    calc_score(target_path, pred_path)

In [13]:
for name in ['msd', 'pearson', 'pearson_baseline']:
    for k in [10, 40, 80]:
        for min_k in [1, 10, 50]:
            print(f'name: {name}, k: {k}, min_k: {min_k}')
            optimize(k=k, 
                     min_k=min_k,
                     user_based=True,
                     name=name,
                     verbose=True)

name: msd, k: 10, min_k: 1
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.0870888233184814 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 100.05it/s]


MRR@100 = 0.0021
name: msd, k: 10, min_k: 10
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.585830450057983 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 191.94it/s]


MRR@100 = 0.0442
name: msd, k: 10, min_k: 50
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.874912261962891 seconds ---


...

MRR@100 = 0.0412
name: msd, k: 80, min_k: 50
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.326446533203125 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 255.53it/s]


MRR@100 = 0.0495
name: pearson, k: 10, min_k: 1
Computing the pearson similarity matrix...
Done computing similarity matrix.
--- fit: 14.512972831726074 seconds ---


 18%|██████████████▏                                                                | 179/1000 [01:34<07:13,  1.89it/s]


KeyboardInterrupt: 

In [15]:
for name in ['msd', 'pearson', 'pearson_baseline']:
    print(f'name: {name}')
    optimize(k=40, 
             min_k=10,
             user_based=True,
             name=name,
             verbose=True)

name: msd
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.944432973861694 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 188.82it/s]


MRR@100 = 0.0478
name: pearson
Computing the pearson similarity matrix...
Done computing similarity matrix.
--- fit: 14.310008764266968 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 283.23it/s]


MRR@100 = 0.0581
name: pearson_baseline
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 13.422325611114502 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 323.47it/s]


MRR@100 = 0.0632


# RATING_PART

In [16]:
%%time

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(data[['uid', 'iid', 'rating_part']], reader)
trainset = dataset.build_full_trainset()

CPU times: total: 875 ms
Wall time: 880 ms


In [17]:
optimize(k=40, 
         min_k=10,
         user_based=True,
         name='pearson_baseline',
         verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 12.321101188659668 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 232.49it/s]


MRR@100 = 0.0612


In [18]:
optimize(k=40, 
         min_k=10,
         user_based=True,
         name='msd',
         verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.0178382396698 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 227.99it/s]


MRR@100 = 0.0377


## Best

In [19]:
%%time

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(data[['uid', 'iid', 'rating_max']], reader)
trainset = dataset.build_full_trainset()

CPU times: total: 1.3 s
Wall time: 2.23 s


In [20]:
for k in [30, 40, 50]:
    for min_k in [5, 10, 20]:
        print(f'k: {k}, min_k: {min_k}')
        optimize(k=k, 
                 min_k=min_k,
                 user_based=True,
                 name='pearson_baseline',
                 verbose=True)

k: 30, min_k: 5
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 36.44635581970215 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 242.03it/s]


MRR@100 = 0.0582
k: 30, min_k: 10
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 27.373706340789795 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 330.27it/s]


MRR@100 = 0.0623
k: 30, min_k: 20
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 10.65767526626587 seconds ---


...

MRR@100 = 0.0577
k: 50, min_k: 10
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 12.935752630233765 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 297.13it/s]


MRR@100 = 0.0637
k: 50, min_k: 20
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 12.497273683547974 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 338.85it/s]

MRR@100 = 0.0556


In [21]:
for k in [50, 75, 100]:
    for min_k in [8, 10, 12]:
        print(f'k: {k}, min_k: {min_k}')
        optimize(k=k, 
                 min_k=min_k,
                 user_based=True,
                 name='pearson_baseline',
                 verbose=True)

k: 50, min_k: 8
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 33.19474816322327 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 292.67it/s]


MRR@100 = 0.0609
k: 50, min_k: 10
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 12.79889726638794 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 330.28it/s]


MRR@100 = 0.0637
k: 50, min_k: 12
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 11.783798456192017 seconds ---


...

MRR@100 = 0.0606
k: 100, min_k: 10
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 11.85500717163086 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 264.57it/s]


MRR@100 = 0.0638
k: 100, min_k: 12
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 12.043147802352905 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 331.64it/s]


MRR@100 = 0.0627


## Final best
k: 75, min_k: 10<br>
Estimating biases using als...<br>
Computing the pearson_baseline similarity matrix...<br>
Done computing similarity matrix.<br>
--- fit: 11.22281551361084 seconds ---<br>
100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:01<00:00,  2.37it/s]<br>
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 332.04it/s]<br>
MRR@100 = 0.0641